In [ ]:
# select = Select(driver.find_element_by_xpath(r'htmlcodehere'))

# select.select_by_index(0)         # selects 0th option from the drop down menu of this xpathed element

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import string
import time
import sqlite3

# pd.set_option('max_columns', None)

In [3]:
driver = webdriver.Chrome()

In [4]:
base_url = 'https://www.basketball-reference.com'
url = r'https://www.basketball-reference.com/leagues/NBA_2018_games.html'
season = '1718'
season_gamecount = 0

In [5]:
# ping1
driver.get(url)

## get boxscore urls for october

In [6]:
src = driver.page_source
parser = BeautifulSoup(src, 'lxml')
table = parser.find('div', attrs = {'class': 'table_container is_setup'})
headers = table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
oct_17_18_url_list = [base_url + header.a['href'] for header in headers]

## get individual boxscores and game info:

In [7]:
# ping2
driver.get(oct_17_18_url_list[0])

In [8]:
src = driver.page_source
parser = BeautifulSoup(src, 'lxml')

stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})

In [9]:
def create_game_info(url, season, season_gamecount):
    
    season_gamecount += 1
    game_count = str(season_gamecount)
    while len(game_count) < 4:
        game_count = '0' + game_count
    
    id_string = url.strip(string.ascii_letters+string.punctuation)
    year = id_string[0:4]
    month = id_string[4:6]
    day = id_string[6:8]
    
    date = year+'-'+month+'-'+day
    
    game_id = int(season+month+day+game_count)
    season = int(season)
    
    return [game_id, season, date]

In [10]:
game_info = create_game_info(oct_17_18_url_list[0], season=season, season_gamecount=season_gamecount)
game_id = game_info[0]

In [15]:
game_info

[171810170001, 1718, '2017-10-17']

In [16]:
def create_team_info(table):
    '''
    Create a dataframe with game results. Uses an html table as input.
    
    ---
    Inputs:
    
    table: a BeautifulSoup html table
    ---
    Outputs:
    
    team_info: a dataframe with the relevant game information (team_ids, scores, and boolean 'results' column)
    '''
    
    # get team_ids
    id_rows = table.findAll('th', attrs={'class':'center', 'data-stat':'team', 'scope':'row'})
    team_ids = [row.text.strip() for row in id_rows]
    
    # get final score
    scores = table.findAll('td', attrs={'class': 'center', 'data-stat': 'T'})
    final_scores = [int(score.text.strip()) for score in scores]
    
    # boolean game-winner: away=0, home=1
    if final_scores[0] > final_scores[1]:
        result=0
    else:
        result=1
    
    team_info = [team_ids[0], final_scores[0], team_ids[1], final_scores[1], result]
    
    return team_info

In [17]:
team_info = create_team_info(id_table)
team_info

['BOS', 99, 'CLE', 102, 1]

In [18]:
team_ids = [team_info[0], team_info[2]]

In [19]:
info_columns = ['game_id', 'season', 'date', 'away_team', 'away_score', 'home_team', 'home_score', 'result']
def create_info_df(game_info, team_info):
    info = game_info + team_info
    info_df = pd.DataFrame([info], columns=info_columns)
    return info_df

In [20]:
info_df = create_info_df(game_info=game_info, team_info=team_info)

## game info df

In [21]:
info_df

,game_id,season,date,away_team,away_score,home_team,home_score,result
0,171810170001,1718,2017-10-17,BOS,99,CLE,102,1


In [22]:
info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   game_id     1 non-null      int64 
 1   season      1 non-null      int64 
 2   date        1 non-null      object
 3   away_team   1 non-null      object
 4   away_score  1 non-null      int64 
 5   home_team   1 non-null      object
 6   home_score  1 non-null      int64 
 7   result      1 non-null      int64 
dtypes: int64(5), object(3)
memory usage: 196.0+ bytes


In [23]:
def create_boxscores(table, game_id):

    # ignore first 'tr', it is table title, not column
    rows = table.findAll('tr')[1:]
    # first 'th' is 'Starters', but will be changed into the player names
    headers = rows[0].findAll('th')
    # provide column names
    headerlist = [h.text.strip() for h in headers]
    
    # ignore first row (headers)
    data = rows[1:]
    # get names column
    player_names = [row.find('th').text.strip() for row in rows]
    # get player stats
    player_stats = [[stat.text.strip() for stat in row.findAll('td')] for row in data]
    # add player name as first entry in each row
    for i in range(len(player_stats)):
        # ignore header with i+1
        player_stats[i].insert(0, player_names[i+1])
    
    # create player stats dataframe
    player_box_df = pd.DataFrame(player_stats, columns=headerlist)
    # drop 'Reserves' row
    player_box_df.drop(player_box_df[player_box_df['Starters'] == 'Reserves'].index, inplace=True)
    
    # add game id column
    player_box_df.insert(loc=0, column='game_id', value=game_id)
    
    # create team stats dataframe from last row in player stats
    team_box_df = pd.DataFrame(player_box_df.iloc[-1]).T
    
    #drop team totals from player stats df
    player_box_df = player_box_df[:-1].rename(columns={'Starters': 'player'})

    return player_box_df, team_box_df

In [24]:
player_box_list = [None, None, None, None]
team_box_list = [None, None, None, None]

for i in range(len(stat_tables)):
    player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

In [25]:
def merge_boxscores(boxscore_list, team_ids, scope):

    # create tuple for every 2 boxscores in list
    pairs = [((boxscore_list[i]), (boxscore_list[i + 1])) for i in range(0, len(boxscore_list), 2)]
    
    clean_boxscores= []
    
    for pair in pairs:
        
        # combine regular and adv boxscores
        df = pd.concat([*pair], axis=1)
        # drop columns with duplicate names
        df = df.loc[:,~df.columns.duplicated()].copy()
        
        clean_boxscores.append(df)
    
    for i in range(len(clean_boxscores)):
        
        if scope=='team':
            clean_boxscores[i].rename(columns={'Starters': 'team'}, inplace=True)
            clean_boxscores[i]['team'] = team_ids[i]
            
        elif scope=='player':
            clean_boxscores[i].insert(loc=2, column='team', value=team_ids[i])
    
    return clean_boxscores

In [26]:
away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')

In [29]:
# team_boxes = away_team_box.append(home_team_box)
team_boxes = pd.concat([away_team_box, home_team_box])
team_boxes.reset_index(drop=True, inplace=True)

In [36]:
team_boxes

,game_id,team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,171810170001,BOS,240,36.0,88.0,0.409,8.0,32.0,0.250,19.0,...,80.4,47.9,66.7,11.1,6.6,9.2,100.0,99.7,102.7,NaN
1,171810170001,CLE,240,38.0,83.0,0.458,5.0,22.0,0.227,21.0,...,82.0,52.1,50.0,3.0,7.1,15.3,100.0,102.7,99.7,NaN


In [32]:
num_columns = ['FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-',
               'FG%', '3P%', 'FT%', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']

In [33]:
def change_dtypes(df, num_columns):

    df.replace(to_replace='', value='-99', inplace=True)
    
    for column in num_columns:
        df[column] = df[column].astype('float64')
        
    df.replace(to_replace=-99, value=np.nan, inplace=True)
    
    return df

## Finished teams box score

In [35]:
team_boxes = change_dtypes(team_boxes, num_columns)

In [37]:
away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')

In [38]:
# player_boxes = away_player_box.append(home_player_box)
player_boxes = pd.concat([away_player_box, home_player_box])
player_boxes.reset_index(drop=True, inplace=True)

In [39]:
player_boxes

,game_id,player,team,MP,FG,FGA,FG%,3P,3PA,3P%,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,171810170001,Jaylen Brown,BOS,39:36,11,23,.478,2,9,.222,...,13.2,7.6,0.0,2.4,0.0,11.2,29.9,89,104,-3.1
1,171810170001,Kyrie Irving,BOS,39:21,8,17,.471,4,9,.444,...,5.3,5.1,46.5,3.7,0.0,10.1,22.2,124,103,10.5
2,171810170001,Jayson Tatum,BOS,36:32,5,12,.417,1,2,.500,...,17.1,13.7,13.4,0.0,0.0,7.0,17.3,115,108,-1.3
3,171810170001,Al Horford,BOS,32:07,2,7,.286,0,2,.000,...,22.7,10.9,22.6,0.0,2.5,0.0,13.8,111,105,0.2
4,171810170001,Gordon Hayward,BOS,5:15,1,2,.500,0,1,.000,...,19.9,9.5,0.0,0.0,0.0,0.0,16.8,95,107,-7.4
5,171810170001,Marcus Smart,BOS,35:03,5,16,.313,0,4,.000,...,26.8,12.8,14.1,2.8,4.5,10.4,24.3,70,96,-4.0
6,171810170001,Terry Rozier,BOS,19:32,2,6,.333,1,3,.333,...,16.0,7.7,15.8,9.9,0.0,0.0,17.5,129,87,14.7
7,171810170001,Aron Baynes,BOS,19:06,2,2,1.000,0,0,,...,16.4,13.1,8.1,0.0,4.1,34.7,13.3,110,105,-4.5
8,171810170001,Semi Ojeleye,BOS,8:39,0,2,.000,0,1,.000,...,0.0,0.0,0.0,0.0,0.0,0.0,10.2,0,112,-17.3
9,171810170001,Shane Larkin,BOS,4:49,0,1,.000,0,1,.000,...,21.7,10.4,0.0,0.0,0.0,0.0,9.1,0,107,-15.0


In [40]:
player_boxes = change_dtypes(player_boxes, num_columns)

In [42]:
totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
totals

{'FG': 74.0,
 'FGA': 171.0,
 'FG%': 0.867,
 '3P': 13.0,
 '3PA': 54.0,
 '3P%': 0.477,
 'FT': 40.0,
 'FTA': 50.0,
 'FT%': 1.6,
 'ORB': 18.0,
 'DRB': 78.0,
 'TRB': 96.0,
 'AST': 43.0,
 'STL': 14.0,
 'BLK': 8.0,
 'TOV': 27.0,
 'PF': 49.0,
 'PTS': 201.0}

In [ ]:
# totals = pd.DataFrame(team_boxes.loc[:,'FG':'PTS'].sum()).T
# totals

In [45]:
def PIE(player_boxes, totals):
    
    PIE_denom = (totals['PTS'] + totals['FG'] + totals['FT'] - totals['FGA'] - totals['FTA'] + totals['DRB'] + (0.5*totals['ORB']) + totals['AST'] + totals['STL'] + (0.5*totals['BLK']) - totals['PF'] - totals['TOV'])
    player_boxes['PIE'] = round((100 * (player_boxes['PTS'] + player_boxes['FG'] + player_boxes['FT'] - player_boxes['FGA'] - player_boxes['FTA'] + player_boxes['DRB'] + (0.5*player_boxes['ORB']) + player_boxes['AST'] + player_boxes['STL'] + (0.5*player_boxes['BLK']) - player_boxes['PF'] - player_boxes['TOV']) / PIE_denom), 1)
    
    return player_boxes

In [ ]:
# PIE_denom = (totals['PTS'] + totals['FG'] + totals['FT'] - totals['FGA'] - totals['FTA'] + totals['DRB'] + (0.5*totals['ORB']) + totals['AST'] + totals['STL'] + (0.5*totals['BLK']) - totals['PF'] - totals['TOV'])

# player_boxes['PIE'] = 100 * (player_boxes['PTS'] + player_boxes['FG'] + player_boxes['FT'] - player_boxes['FGA'] - player_boxes['FTA'] + player_boxes['DRB'] + (0.5*player_boxes['ORB']) + player_boxes['AST'] + player_boxes['STL'] + (0.5*player_boxes['BLK']) - player_boxes['PF'] - player_boxes['TOV']) / PIE_denom

In [48]:
player_boxes = PIE(player_boxes, totals)
player_boxes

,game_id,player,team,MP,FG,FGA,FG%,3P,3PA,3P%,...,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,PIE
0,171810170001,Jaylen Brown,BOS,39:36,11.0,23.0,0.478,2.0,9.0,0.222,...,7.6,0.0,2.4,0.0,11.2,29.9,89.0,104.0,-3.1,6.9
1,171810170001,Kyrie Irving,BOS,39:21,8.0,17.0,0.471,4.0,9.0,0.444,...,5.1,46.5,3.7,0.0,10.1,22.2,124.0,103.0,10.5,13.9
2,171810170001,Jayson Tatum,BOS,36:32,5.0,12.0,0.417,1.0,2.0,0.500,...,13.7,13.4,0.0,0.0,7.0,17.3,115.0,108.0,-1.3,7.8
3,171810170001,Al Horford,BOS,32:07,2.0,7.0,0.286,0.0,2.0,0.000,...,10.9,22.6,0.0,2.5,0.0,13.8,111.0,105.0,0.2,7.5
4,171810170001,Gordon Hayward,BOS,5:15,1.0,2.0,0.500,0.0,1.0,0.000,...,9.5,0.0,0.0,0.0,0.0,16.8,95.0,107.0,-7.4,0.6
5,171810170001,Marcus Smart,BOS,35:03,5.0,16.0,0.313,0.0,4.0,0.000,...,12.8,14.1,2.8,4.5,10.4,24.3,70.0,96.0,-4.0,6.6
6,171810170001,Terry Rozier,BOS,19:32,2.0,6.0,0.333,1.0,3.0,0.333,...,7.7,15.8,9.9,0.0,0.0,17.5,129.0,87.0,14.7,8.4
7,171810170001,Aron Baynes,BOS,19:06,2.0,2.0,1.000,0.0,0.0,NaN,...,13.1,8.1,0.0,4.1,34.7,13.3,110.0,105.0,-4.5,1.5
8,171810170001,Semi Ojeleye,BOS,8:39,0.0,2.0,0.000,0.0,1.0,0.000,...,0.0,0.0,0.0,0.0,0.0,10.2,0.0,112.0,-17.3,-1.8
9,171810170001,Shane Larkin,BOS,4:49,0.0,1.0,0.000,0.0,1.0,0.000,...,10.4,0.0,0.0,0.0,0.0,9.1,0.0,107.0,-15.0,0.0


In [ ]:
# ping 2
src = driver.page_source
parser = BeautifulSoup(src, 'lxml')
months = parser.find('div', attrs = {'class': 'filter'})
links = months.findAll('a')
month_links = [base_url + link['href'] for link in links]

month_links

links[0]['href']

driver.find_element(By.LINK_TEXT, 'October')

driver.find_element(By.LINK_TEXT, 'October').click();

months = ['October', 'November', 'December', 'January', 'February', 'March', 'April']

In [ ]:
conn = sqlite3.connect('test_database') 
c = conn.cursor()

In [ ]:
away_player_box.to_sql('player_stats', con=conn, if_exists='append', index=False)

In [ ]:
home_player_box.to_sql('player_stats', con=conn, if_exists='append', index=False)

In [ ]:
query = "SELECT * from player_stats"

# pd.read_sql_query(query, conn, index_col='index')
# df.index.names = ['']

In [ ]:
dffd = pd.read_sql_query(query, conn, index_col='index')
dffd

In [ ]:
conn.close()

In [ ]:
driver.quit()